In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import matplotlib as plt
%matplotlib inline
import time

In [2]:
link='https://www.indeed.com/jobs?q=data+scientist+%24120000&l=Sunnyvale%2C+CA'

In [3]:
#making a list of cities to run through
cities = ['Redwood+City', 'Menlo+Park', 'San+Jose', 'Milpitas', 'Sunnyvale', 'Santa+Clara']

In [4]:

### Rithika suggested I merge all of the find_alls into one large function, so the code below was created with her assistance. 

def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text.text.replace('\n', '')
        except:
            salary = 'None'
        synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
    return df

In [5]:
parse(link)

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:

url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 200 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
i = 0
results = []
df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
for city in cities:
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = url_template.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text.replace('\n', '')
            except:
                salary = None
            try:
                synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            except:
                synopsis = None
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            i += 1
            if i % 1000 == 0:  # Ram helped me build this counter to see how many. You can visibly see Ram's vernacular in the print statements.
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")

In [ ]:
df_more.to_csv('Indeed_jobs_near_me.csv', encoding='utf-8')

In [ ]:
df_more = pd.read_csv('C:/Users/Ghezae/Indeed_jobs_near_me.csv')

In [ ]:
df_more.drop('Unnamed: 0', axis =1, inplace=True)

In [ ]:
df_more.head()

In [ ]:
print(df_more[df_more.Salary != 'None'].shape)

In [ ]:
df_more = df_more[df_more.Salary !='None'].drop_duplicates().dropna()

In [ ]:
df_more.head()